## Report for Ed Tech Company

In [3]:
import json
from pyspark.sql import Row
import pprint

In [4]:
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 
assessments = raw_assessments.select(raw_assessments.value.cast('string'))
extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()
extracted_assessments.registerTempTable('assessments')

We get our messages from the 'assessments' kafka topic and transform values into strings. We then extract the data from the 'value' portion of our message and create a data frame.

In [5]:
assessments_as_strings =raw_assessments.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

To further understand what an individual message's contents look like, we cast the assessments as strings and visualize one of them.

In [6]:
first_assessment=json.loads(assessments_as_strings.select('value').take(1)[0].value)

In [7]:
first_assessment

{'base_exam_id': '37f0a30a-7464-11e6-aa92-a8667f27e5dc',
 'certification': 'false',
 'exam_name': 'Normal Forms and All That Jazz Master Class',
 'keen_created_at': '1516717442.735266',
 'keen_id': '5a6745820eb8ab00016be1f1',
 'keen_timestamp': '1516717442.735266',
 'max_attempts': '1.0',
 'sequences': {'attempt': 1,
  'counts': {'all_correct': False,
   'correct': 2,
   'incomplete': 1,
   'incorrect': 1,
   'submitted': 4,
   'total': 4,
   'unanswered': 0},
  'id': '5b28a462-7a3b-42e0-b508-09f3906d1703',
  'questions': [{'id': '7a2ed6d3-f492-49b3-b8aa-d080a8aad986',
    'options': [{'at': '2018-01-23T14:23:24.670Z',
      'checked': True,
      'correct': True,
      'id': '49c574b4-5c82-4ffd-9bd1-c3358faf850d',
      'submitted': 1},
     {'at': '2018-01-23T14:23:25.914Z',
      'checked': True,
      'correct': True,
      'id': 'f2528210-35c3-4320-acf3-9056567ea19f',
      'submitted': 1},
     {'checked': False,
      'correct': True,
      'id': 'd1bf026f-554f-4543-bdd2-54dcf10

As we see, the message's contents have a complicated nested schema containing a variety of information relating to the exam and its questions.

In order to get an idea of what the data looks like, we run the following simple queries.

### How many assessments are in the dataset?

In [8]:
spark.sql("select count(*) from assessments").show()

+--------+
|count(1)|
+--------+
|    3280|
+--------+



There are 3280 assessments

### How many people took Learning Git?


In [7]:
spark.sql("select count(*) from assessments where exam_name = 'Learning Git'").show()

+--------+
|count(1)|
+--------+
|     394|
+--------+



394 people took Learning Git

### What is the least common course taken? And the most common?


In [9]:
spark.sql("select count(*) as ct, exam_name from assessments group by exam_name order by ct asc ").show(4, truncate=False)

spark.sql("select count(*) as ct, exam_name from assessments group by exam_name order by ct desc").show(2,truncate=False)

+---+-------------------------------------------------+
|ct |exam_name                                        |
+---+-------------------------------------------------+
|1  |Nulls, Three-valued Logic and Missing Information|
|1  |Native Web Apps for Android                      |
|1  |Learning to Visualize Data with D3.js            |
|1  |Operating Red Hat Enterprise Linux Servers       |
+---+-------------------------------------------------+
only showing top 4 rows

+---+----------------------+
|ct |exam_name             |
+---+----------------------+
|394|Learning Git          |
|162|Introduction to Python|
+---+----------------------+
only showing top 2 rows



There are 4 exams sharing the last place, all having 1 instance. On the other hand, the "Learning Git" class has a firm grip on the first place, superceeding the next class by over 200 instances.

## Business Questions

### What is the average score for each class?

Perhaps it would yield useful insight regarding an exam's difficulty level to know what its average score is. To more easily deal with this data, I adapt a lambda function to extract from the nested .json a series of columns relating to  exams and their questions. These columns are the exam's name, the total number of questions, the number of correct responses and the number of incomplete responses.

In [81]:
def my_lambda_correct_total(x):
    
    raw_dict = json.loads(x.value)
    my_list = []
    
    if "sequences" in raw_dict:
        
        if "counts" in raw_dict["sequences"]:
            
            if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
                    
                my_dict = {"correct": raw_dict["sequences"]["counts"]["correct"], 
                           "total": raw_dict["sequences"]["counts"]["total"],
                          "exam": raw_dict["exam_name"],
                          "incomplete": raw_dict["sequences"]["counts"]["incomplete"]}
                my_list.append(Row(**my_dict))
    
    return my_list

my_correct_total = assessments.rdd.flatMap(my_lambda_correct_total).toDF()

my_correct_total.registerTempTable('ct')

All of this data is registered to a temporary table by the name of 'ct' for "counts".

From this table, I firstly make use of the exam name, the correct, and the total questions counts. Below, I write an SQL query that computes the mean percentage score for each exam and saves it in a means variable. This variable is then saved and made available through parquet.

In [82]:
means = spark.sql("select exam, mean(percentage) as mean from (select exam, correct, total, correct/total as percentage from ct order by exam) group by exam order by mean desc")

In [83]:
means.show(truncate=False)

+-----------------------------------------------------------------------+------------------+
|exam                                                                   |mean              |
+-----------------------------------------------------------------------+------------------+
|Nulls, Three-valued Logic and Missing Information                      |1.0               |
|Learning to Visualize Data with D3.js                                  |1.0               |
|The Closed World Assumption                                            |1.0               |
|Learning SQL for Oracle                                                |0.9772727272727273|
|Introduction to Java 8                                                 |0.8759493670886073|
|Introduction to Amazon Web Services (AWS) - EC2 Deployment Fundamentals|0.8333333333333334|
|Introduction to Apache Spark                                           |0.8333333333333334|
|Cloud Native Architecture Fundamentals                               

In [84]:
means.write.mode('overwrite').parquet("/tmp/means")

### For each exam, when was the earliest and latest start date and how many people took it?

Perhaps, another question of interest would be the time frame which the exams in our dataset took place. For each class, we see when the earliest and latest exam instances were as well as the number of instances that took place. The SQL query extracts dates from the 'started_at' timestamps in our data.

As above, the data is saved to a variable and fed through parquet.

In [10]:
start_dates = spark.sql("select CAST(min(started_at) as DATE) as earliest, CAST(max(started_at) as DATE) as latest, exam_name as exam, count(exam_name) as count from assessments group by exam_name order by earliest")

In [110]:
start_dates.show(truncate = False)

+----------+----------+--------------------------------------------------------------+-----+
|earliest  |latest    |exam                                                          |count|
+----------+----------+--------------------------------------------------------------+-----+
|2017-11-21|2018-01-25|Cloud Computing With AWS                                      |17   |
|2017-11-21|2018-01-26|Advanced Machine Learning                                     |67   |
|2017-11-21|2018-01-09|Collaborating with Git                                        |6    |
|2017-11-21|2018-01-23|Git Fundamentals for Web Developers                           |28   |
|2017-11-21|2018-01-28|Software Architecture Fundamentals Beyond The Basics          |48   |
|2017-11-21|2018-01-25|Introduction to Machine Learning                              |119  |
|2017-11-21|2018-01-25|Practical Java Programming                                    |53   |
|2017-11-21|2018-01-28|Software Architecture Fundamentals Understandin

In [35]:
start_dates.write.mode('overwrite').parquet("/tmp/start_dates")

### What is the average number of incomplete questions for every exam and how does this number compare to the total number of questions?

Often times, when users find a question to be too challenging they leave it unanswered. Another way to gauge the difficulty level of an exam is to get an idea of how many questions it has and see the average number of questions left unanswered. The query below does exactly this as well as compute the proportional size of the set of unanswered questions with respect to the total number of questions.

In [106]:
incomplete = spark.sql("select exam, max(total) as total, mean(incomplete) as incomplete_average, mean(incomplete)/mean(total) as proportional from ct group by exam order by incomplete_average desc")

In [107]:
incomplete.show(truncate=False)

+----------------------------------------------------+-----+------------------+-------------------+
|exam                                                |total|incomplete_average|proportional       |
+----------------------------------------------------+-----+------------------+-------------------+
|Arduino Prototyping Techniques                      |6    |2.0               |0.3333333333333333 |
|Introduction to Data Science with R                 |7    |1.6744186046511629|0.23920265780730898|
|Building Web Services with Java                     |4    |1.6666666666666667|0.4166666666666667 |
|I'm a Software Architect, Now What?                 |4    |1.4666666666666666|0.36666666666666664|
|Cloud Computing With AWS                            |4    |1.411764705882353 |0.35294117647058826|
|Mastering Web Views                                 |4    |1.3333333333333333|0.3333333333333333 |
|Learning Java EE 7                                  |3    |1.2               |0.39999999999999997|


In [76]:
incomplete.write.mode('overwrite').parquet("/tmp/incomplete")

### Relevant Commands from `philpapapolyzos-history.txt`

```
cd project-2-philpapapolyzos/

docker-compose up -d

docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --
replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

docker-compose exec mids bash -c "cat /w205/project-2-philpapapolyzos/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments"  

docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark
```